In [4]:
library(plyr)
library(tidyverse)
library(infer)
library(janitor)
library(repr)
library(stringr)
library(ggthemes)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [5]:
url <- "https://geodash.vpd.ca/opendata/crimedata_download/crimedata_csv_all_years.zip"
filename <- "crime_data.zip"
download.file(url, destfile = filename)

# Data comes in as a zip, so we'll need to extract it
unzip("crime_data.zip")

# Read in the desired file
crime_data <- read_csv("crimedata_csv_all_years.csv")



── Column specification ────────────────────────────────────────────────────────
cols(
  TYPE = col_character(),
  YEAR = col_double(),
  MONTH = col_double(),
  DAY = col_double(),
  HOUR = col_double(),
  MINUTE = col_double(),
  HUNDRED_BLOCK = col_character(),
  NEIGHBOURHOOD = col_character(),
  X = col_double(),
  Y = col_double()
)




In [150]:
theft_crimes <- c("Other Theft", "Theft from Vehicle", 
                  "Theft of Bicycle", "Theft of Vehicle")
crime_data <- crime_data %>% clean_names() %>% mutate(is_theft = type%in% theft_crimes)

In [148]:
shaughnessy <- crime_data %>% filter(neighbourhood=="Shaughnessy")
strathcona <- crime_data %>% filter(neighbourhood=="Strathcona")
marpole <- crime_data %>% filter(neighbourhood=="Marpole")

[1] 6782

[1] 16202

[1] 28065

In [60]:
pre_years <- c(2017, 2018, 2019)
shaughnessy <- shaughnessy %>%
    select(is_theft, year) %>%
    filter(year >= 2017) %>%
    mutate(period=ifelse(year %in% pre_years, 'pre_covid', 'covid')) %>%
    mutate(period=as.factor(period))


In [61]:
head(shaughnessy)

is_theft,year,period
<lgl>,<dbl>,<fct>
FALSE,2018,pre_covid
FALSE,2019,pre_covid
FALSE,2021,covid
FALSE,2020,covid
FALSE,2018,pre_covid
FALSE,2020,covid


In [67]:
pre_covid <- shaughnessy %>%
    filter(period=="pre_covid")
pre_covid_observed <- mean(as.numeric(pre_covid$is_theft))

covid <- shaughnessy %>%
    filter(period=="covid") 
covid_observed <- mean(as.numeric(covid$is_theft))

observed_diff_in_props <- covid_observed - pre_covid_observed
observed_diff_in_props

[1] -0.03512647

In [73]:
shaughnessy_infer <- shaughnessy %>%
    specify(is_theft ~ period, success="TRUE") %>%
    hypothesise(null="independence") %>%
    generate(reps=10000, type="permute") %>%
    calculate(stat="diff in props", order=c("covid", "pre_covid"))

In [133]:
shaughnessy_infer %>% get_p_value(observed_diff_in_props, direction="two_sided") %>% pull()

[1] 0.234

In [165]:
neighbourhoods <- unique(crime_data$neighbourhood)
neighbourhoods <- tibble(neighbourhood=as.character(neighbourhoods[!is.na(neighbourhoods)]))
columns <- c("neighbourhood", "past_prop", "current_prop", "diff_in_props", "p_value")

In [167]:
crime_data_processed <- crime_data %>%
    select(neighbourhood, year, is_theft) %>%
    filter(year >= 2017) %>%
    mutate(period=ifelse(year %in% pre_years, 'pre_covid', 'covid')) %>%
    mutate(period=as.factor(period)) %>%
    mutate(neighbourhood=neighbourhood)
head(crime_data_processed)

neighbourhood,year,is_theft,period
<chr>,<dbl>,<lgl>,<fct>
Fairview,2019,FALSE,pre_covid
West End,2019,FALSE,pre_covid
West End,2020,FALSE,covid
West End,2017,FALSE,pre_covid
West End,2018,FALSE,pre_covid
West End,2020,FALSE,covid


In [175]:
results <- data.frame(matrix(ncol=length(columns), nrow=0))
names(results) <- columns

In [176]:
set.seed(12345)
for (i in 1:nrow(neighbourhoods)) {
    n <- neighbourhoods$neighbourhood[i]
    neighbourhood_data <- crime_data_processed %>% filter(neighbourhood==n)
    
    pre_covid <- neighbourhood_data %>%
        filter(period=="pre_covid")
    pre_covid_prop <- mean(as.numeric(pre_covid$is_theft))
    
    covid <- neighbourhood_data %>%
        filter(period=="covid")
    covid_prop <- mean(as.numeric(covid$is_theft))
    
    obs_diff_in_props <- covid_prop - pre_covid_prop
    
    neighbourhood_infer <- neighbourhood_data %>%
        specify(is_theft ~ period, success="TRUE") %>%
        hypothesise(null="independence") %>%
        generate(reps=500, type="permute") %>%
        calculate(stat="diff in props", order=c("covid", "pre_covid"))
    
    p_value <- neighbourhood_infer %>% get_p_value(observed_diff_in_props, direction="two_sided") %>% pull()
    
    new_obs <- data.frame(n, pre_covid_prop, covid_prop, obs_diff_in_props, p_value)
    names(new_obs) <- columns
    results <- rbind(results, new_obs)
}

Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()` for more information.”
Warning message:
“Please be cautious in reporting a p-value of 0. This result is an approximation based on the number of `reps` chosen in the `generate()` step. See `?get_p_value()

In [178]:
results <- results %>%
    mutate(reject5=p_value < 0.05, reject10=p_value < 0.10)
results

neighbourhood,past_prop,current_prop,diff_in_props,p_value,reject5,reject10
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
Oakridge,0.5432873,0.4933687,-0.049918627,0.244,FALSE,FALSE
Fairview,0.6891537,0.6078619,-0.081291734,0.000,TRUE,TRUE
West End,0.7351588,0.6428571,-0.092301674,0.000,TRUE,TRUE
Central Business District,0.7533982,0.5980081,-0.155390135,0.000,TRUE,TRUE
Hastings-Sunrise,0.6451613,0.5623028,-0.082858451,0.016,TRUE,TRUE
Strathcona,0.5577118,0.4352824,-0.122429420,0.004,TRUE,TRUE
Grandview-Woodland,0.6237392,0.5418350,-0.081904225,0.004,TRUE,TRUE
Kitsilano,0.6463512,0.6131550,-0.033196282,0.012,TRUE,TRUE
Kensington-Cedar Cottage,0.6043016,0.6066754,0.002373820,0.012,TRUE,TRUE
